In [1]:
import os
import sys

path_root = os.path.dirname(os.getcwd())

if path_root not in sys.path:
    sys.path.append(path_root)

In [2]:
from src.config import (
    path_output_synthea,
    path_output_llm,
    path_output_extraction,
    path_output_standardisation,
)
from src.generate.synthea import GenerateSynthea
from src.generate.llm import GenerateLLM
from src.extraction.extraction import Extraction
from src.standardise_extraction.standardise_extraction import (
    StandardiseExtraction,
)
from src.privacy_risk_scorer.privacy_risk_scorer import PrivacyRiskScorer
from src.privacy_risk_explainer.privacy_risk_explainer import (
    PrivacyRiskExplainer,
)

In [3]:
import spacy
from spacy import displacy
import ipywidgets as widgets
from IPython.display import display, HTML

In [4]:
fake_data = [
    "Clinical Note:\nPatient: Harris Fadel\nNHS Number: 5927778917\nDate of Birth: August 8, 2005\n\nPresentation: Acute bronchitis (disorder)\n\nSymptoms:\n\n* Cough\n* Chest tightness\n* Shortness of breath\n\nMedical History:\n\n* Recent respiratory infection\n\nReview of Systems:\n\n* No reported fever or chills\n* No cough productive of yellow or green mucus\n\nPlan:\n\n* Prescribe antibiotics for 7-10 days\n* Instruct patient to rest and avoid strenuous activities\n* Monitor patient's condition closely and adjust plan as needed.",
    "Clinical Note for Patient Wallace Tremblay:\n\n Patient with lime green curly hair presents with acute bronchitis, as confirmed by clinical evaluation and laboratory results. The patient is a 13-year-old male with a NHS number of 5693788113, born on June 13th, 2006. Symptoms include cough, chest tightness, and fever. Treatment plan includes antibiotics and bronchodilators as needed, as well as instructions for self-care and rest. Further evaluation and monitoring are necessary to ensure proper management of this condition.",
    'Certainly! Here is an example clinical note for Shayna VonRueden based on the provided information:\n\n"Patient presents with viral sinusitis. Date of birth: 11/28/2015. NHS number: 0939087782. Given name: Shayna. Family name: VonRueden. Diagnosis: Viral sinusitis (disorder)."\n\nI hope this meets your requirements! Let me know if you have any further questions.',
    "Clinical Note:\nPatient Name: Ramon Kohler\nNHS Number: 0169779904\nDate of Birth: May 9, 1973\n\nPresenting Complaint: Chronic pain in the left shoulder and arm, affecting daily activities.\n\nHistory of Present Illness: The patient experienced a fall two months ago resulting in a dislocation of the left shoulder. Since then, the patient has been experiencing persistent pain in the left shoulder and arm, which has worsened despite initial treatment with analgesics.\n\nReview of Systems: The patient reports difficulty sleeping due to pain and experiences numbness and tingling in the left arm. There is no history of trauma or injury to the left shoulder or arm.\n\nPlan: Further evaluation and imaging studies are necessary to determine the cause of chronic pain. In the meantime, the patient will be referred to physical therapy for non-invasive treatment options.",
    "Clinical Note:\n\nPatient Name: Tyesha Dach\nNHS Number: 3494867836\nDate of Birth: September 24, 1970\n\nDiagnosis: Childhood asthma\n\nChief Complaint: Wheezing and shortness of breath for the past week.\n\nHistory of Present Illness: The patient is a 5-year-old girl who presented to the clinic with a 2-day history of wheezing and shortness of breath. She was seen in the emergency room earlier in the week for similar symptoms, where she was diagnosed with asthma and prescribed an inhaler. However, her symptoms persisted, leading her parents to seek further evaluation.\n\nSystems Examination: The patient's breath sounds are adventitious, with a high-pitched wheezing sound on expiration. Her chest is slightly expanded, and there is minimal tactile fremitus. The patient's oxygen saturation is 90% on room air.\n\nRecommendations:\n\n* Continue prescribed inhaler therapy\n* Monitor symptoms closely and adjust medication as needed\n* Educate parents on proper use of inhaler and management of asthma symptoms",
    "Clinical Note:\nMr. Austin Morar, a 59-year-old male patient, presents to the clinic today with a 3-day history of cough and fever. He reports difficulty breathing and chest tightness. His NHS number is 7723641893 and he was born on April 7th, 1963. Based on his symptoms and medical history, I diagnose him with acute bronchitis. Further assessment and testing may be necessary to rule out any underlying conditions.",
    "Clinical Note:\nMs. Allison King, a 53-year-old woman, presented to the clinic for her scheduled prenatal appointment at 20 weeks gestation. She is in normal health and has no significant medical history. Her NHS number is 9877791390, she was born on April 6th, 1967, and her diagnosis is normal pregnancy.",
    "Clinical Note:\nPatient Name: Juanita Okuneva\nNHS Number: 5870122848\nDate of Birth: January 22, 1951\n\nChief Complaint: Acute bronchitis (disorder)\n\nHistory of Present Illness: Patient Juanita Okuneva, a 71-year-old female, presents with cough and fever for the past week. She reports difficulty breathing and chest tightness. No recent travel or exposure to sick individuals.\n\nAssessment: Based on patient's symptoms and medical history, acute bronchitis is a likely diagnosis. Further evaluation and diagnostic tests will be conducted to rule out other potential causes of respiratory distress.",
    "Clinical Note for Valda Morissette:\n\nMs. Morissette presents today with persistent perennial allergic rhinitis, characterized by nasal congestion, sneezing, and itchy eyes. She reports worsening symptoms throughout the spring and summer months, consistent with her seasonal allergy triggers. Her NHS number is 6393976098 and she was born on February 2nd, 1962.",
    "Clinical Note:\nMr. John Bosco, a 65-year-old male patient, was admitted to the hospital on March 18th with acute bronchitis. His NHS number is 1954287143 and he was born on March 16th, 1953. His diagnosis is acute bronchitis (disorder).",
]

In [5]:
# The standard list of entities is here
ent_list = ["person", "nhs number", "date of birth", "diagnosis"]

output_extraction = Extraction(
    llm_input=fake_data,
    path_output=path_output_extraction,
    save_output=True,
    ent_list=ent_list,
).run(server_model_type="gliner")

/mnt/c/Users/Will Poulett/Documents/PrivFingerprint/privfp-experiments/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
entity_dicts = []
current_index = 1

for i in range(len(output_extraction)):
    entity_dicts.append(
        {
            "text": fake_data[i],
            "ents": output_extraction[i]["Entities"],
        }
    )

# Define initial widgets
button = widgets.Button(description="Next Note")
output = widgets.Output()
note_input = widgets.Text(placeholder="New Entity type: (e.g. hair colour)")
submit_button = widgets.Button(description="Add Entity")
notes_output = widgets.Output()


# Define a function to update and display the displaCy visualization
def update_visualization(dic):
    with output:
        output.clear_output()
        html = displacy.render(dic, manual=True, style="ent")
        display(HTML(html))


# Define a function to handle button click
def on_button_click(change):
    global current_index
    dic = entity_dicts[current_index]
    update_visualization(dic)
    current_index = (current_index + 1) % len(dic)


# Define a function to handle button click for submitting note
def on_submit_button_click(change):
    note_text = note_input.value.strip()
    if note_text:
        ent_list.append(note_text)
        update_notes_output()


# Define a function to update notes output
def update_notes_output():
    with notes_output:
        notes_output.clear_output()


# Attach the button click event handler
button.on_click(on_button_click)
submit_button.on_click(on_submit_button_click)

input_box = widgets.HBox([note_input, submit_button])
# Display the initial button and output area

display(output)
display(button)
display(input_box)

update_visualization(entity_dicts[0])

Output()

Button(description='Next Note', style=ButtonStyle())

In [7]:
ent_list

['person', 'nhs number', 'date of birth', 'diagnosis', 'hair colour']

In [8]:
output_extraction = Extraction(
    llm_input=fake_data,
    path_output=path_output_extraction,
    save_output=True,
    ent_list=ent_list,
).run(server_model_type="gliner")

In [9]:
entity_dicts = []
current_index = 1

for i in range(len(output_extraction)):
    entity_dicts.append(
        {
            "text": fake_data[i],
            "ents": output_extraction[i]["Entities"],
        }
    )

# Define initial widgets
button = widgets.Button(description="Next Note")
output = widgets.Output()
note_input = widgets.Text(placeholder="New Entity type: (e.g. hair colour)")
submit_button = widgets.Button(description="Add Entity")
notes_output = widgets.Output()

# Attach the button click event handler
button.on_click(on_button_click)
submit_button.on_click(on_submit_button_click)

input_box = widgets.HBox([note_input, submit_button])
# Display the initial button and output area

display(output)
display(button)
display(input_box)

update_visualization(entity_dicts[0])

Output()

Button(description='Next Note', style=ButtonStyle())